# Import namespaces

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split

import pickle

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
from tensorflow.keras import layers

# Load dataset

In [ ]:
# Load the training data into a DataFrame named 'train'. 
# Print the shape of the resulting DataFrame. 
# You do not need the test data in this notebook. 

train = pd.read_csv(f'../input/histopathologic-cancer-detection/train_labels.csv', dtype=str)

print('Training Set Size:', train.shape)

train.head()

In [ ]:
# # Lets play with 1% data to check if all code works
# # Comment this when running the entire code
# ignore, train = train_test_split(train, test_size=0.01, random_state=1, stratify=train.label)
# print('Training Set Size:', train.shape)

Lets update the dataset to include filename extensions

In [ ]:
train['id'] = train['id'].apply(lambda x: f'{x}.tif')
train.head()

# Label Distribution

In [ ]:
(train.label.value_counts() / len(train)).to_frame().sort_index().T

# View Sample of Images

In [ ]:
train_path = "../input/histopathologic-cancer-detection/train"
print('Training Images:', len(os.listdir(train_path)))

sample = train.sample(n=16).reset_index()

plt.figure(figsize=(8,8))

for i, row in sample.iterrows():

    img = mpimg.imread(f'../input/histopathologic-cancer-detection/train/{row.id}')    
    label = row.label

    plt.subplot(4,4,i+1)
    plt.imshow(img)
    plt.text(0, -5, f'Class {label}', color='k')
        
    plt.axis('off')

plt.tight_layout()
plt.show()

# Data Generators

In [ ]:
train_df, valid_df = train_test_split(train, test_size=0.2, random_state=1, stratify=train.label)

print(train_df.shape)
print(valid_df.shape)

In [ ]:
# Create image data generators for both the training set and the validation set. 
# Use the data generators to scale the pixel values by a factor of 1/255. 

train_datagen = ImageDataGenerator(rescale=1/255)
valid_datagen = ImageDataGenerator(rescale=1/255)

In [ ]:
# Complete the code for the data loaders below. 

BATCH_SIZE = 64

train_loader = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    directory = train_path,
    x_col = 'id',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = 1,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (96,96)
)

valid_loader = train_datagen.flow_from_dataframe(
    dataframe = valid_df,
    directory = train_path,
    x_col = 'id',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = 1,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (96,96)
)

In [ ]:
TR_STEPS = len(train_loader)
VA_STEPS = len(valid_loader)

print(TR_STEPS)
print(VA_STEPS)

# Build Network

In [ ]:
SEED = 1

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical", seed=SEED, input_shape=(96,96,3)),
    layers.RandomRotation(0.5, seed=SEED),
    layers.RandomZoom(0.3, 0.3, seed=SEED),
    layers.RandomContrast(0.3, seed=SEED),
    layers.RandomTranslation(0.3, 0.3, seed=SEED)
])


np.random.seed(SEED)
tf.random.set_seed(SEED)

cnn = Sequential([
    
    # Cropping2D(cropping=((32, 32), (32, 32)), input_shape=(96,96,3)),
    
    data_augmentation,
    
    Conv2D(32, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(32, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.3),
    BatchNormalization(),

    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.4),
    BatchNormalization(),

    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.5),
    BatchNormalization(),

    Flatten(),
    
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.4),
    Dense(16, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(2, activation='softmax')
])

cnn.summary()

# Train Network

In [ ]:
# Define an optimizer and select a learning rate. 
# Then compile the model. 

opt = tf.keras.optimizers.Adam(0.001)
cnn.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])

In [ ]:
%%time 

# Complete one or more training runs. 
# Display training curves after each run. 

h1 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 35,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1,
    use_multiprocessing=True, 
    workers=8
)

history = h1.history
print(history.keys())

In [ ]:
epoch_range = range(1, len(history['loss'])+1)

plt.figure(figsize=[14,4])
plt.subplot(1,3,1)
plt.plot(epoch_range, history['loss'], label='Training')
plt.plot(epoch_range, history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()
plt.subplot(1,3,2)
plt.plot(epoch_range, history['accuracy'], label='Training')
plt.plot(epoch_range, history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()
plt.subplot(1,3,3)
plt.plot(epoch_range, history['auc'], label='Training')
plt.plot(epoch_range, history['val_auc'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('AUC'); plt.title('AUC')
plt.legend()
plt.tight_layout()
plt.show()

# Training Run 2

In [ ]:
tf.keras.backend.set_value(cnn.optimizer.learning_rate, 0.0001)

In [ ]:
%%time 

h2 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 35,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1,
    use_multiprocessing=True, 
    workers=8
)

In [ ]:
for k in history.keys():
    history[k] += h2.history[k]

epoch_range = range(1, len(history['loss'])+1)

plt.figure(figsize=[14,4])
plt.subplot(1,3,1)
plt.plot(epoch_range, history['loss'], label='Training')
plt.plot(epoch_range, history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()
plt.subplot(1,3,2)
plt.plot(epoch_range, history['accuracy'], label='Training')
plt.plot(epoch_range, history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()
plt.subplot(1,3,3)
plt.plot(epoch_range, history['auc'], label='Training')
plt.plot(epoch_range, history['val_auc'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('AUC'); plt.title('AUC')
plt.legend()
plt.tight_layout()
plt.show()

# Save Model and History

In [ ]:
cnn.save('cancer_model_v01.h5')
pickle.dump(history, open(f'cancer_history_v01.pkl', 'wb'))